[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

**Author and Copyright Notice** *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*

In [2]:
# library import
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [3]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [4]:
# Load initial json file
with open('timeseries.json','r') as f:
    data = json.loads(f.read())

In [6]:
# Reformat (Wrangle) data into a more workble format
def wrangle_data(data):
    
    df = pd.json_normalize(data, record_path =['data'])
 
    # Reverse order of data (Start with the oldest one)
    df = df.loc[::-1].reset_index(drop=True)
    # Fill in NAN value with 0.0
    df.fillna(0.0, inplace=True)
    
    return df

df = wrangle_data(data)

            date  cases  hospital
0     2020-01-31      2       0.0
1     2020-02-01      0       0.0
2     2020-02-02      0       0.0
3     2020-02-03      0       0.0
4     2020-02-04      0       0.0
...          ...    ...       ...
1024  2022-11-20      0     434.0
1025  2022-11-21      0     503.0
1026  2022-11-22      0       0.0
1027  2022-11-23      0       0.0
1028  2022-11-24  20261       0.0

[1029 rows x 3 columns]


In [12]:
#Download the latest data by accessing the API.
def access_api():
    """ Access the PHE API. Return raw data in the same format as data loaded from the "canned" JSON file. """
    england_only = [
        'areaType=nation',
        'areaName=England'
    ]

    cases_death_and_vaccin = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",

    }

    api = Cov19API(
        filters=england_only,
        structure=cases_death_and_vaccin,
    )

    data = api.get_json()
    
    return data

In [7]:
#Set a button to refresh data and redraw the graph
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. 
    apidata = access_api()
    
    # Wrangle the data and overwrite the dataframe for plotting
    global df
    df = wrangle_data(apidata)
    
    # Redraw the graph with lastest data access from API
    refresh_graph()
    
    # Change the status of the button
    apibutton.icon="check"

    
apibutton=wdg.Button(
    description='REFRESH',
    disabled=False,
    button_style='',
    tooltip="Keep calm and carry on",
    icon='rotate-right'
)

# Register button callback function with the button
apibutton.on_click(api_button_callback)

display(apibutton)

Button(description='REFRESH', icon='rotate-right', style=ButtonStyle(), tooltip='Keep calm and carry on')

## Comparison of the number of cases with hospital admissions;

In [8]:
# Fifth Step: Display the interactive graph
def graph(graphcolumns):
    plt.plot(df['date'], df[graphcolumns])
    plt.xticks(np.arange(50, 1031, 60), rotation=90)
    plt.show()
    
# Make it a drop down    
cols = wdg.Dropdown(
    options=['cases', 'hospital'],
    value='cases',
    description='Overview: ',
    disabled=False
)

# Refresh function
def refresh_graph():
    """ Change the value of the widget in order to force a redraw of the graph."""
    current=cols.value
    if current==cols.options[0]:
        other=cols.options[1]
    else:
        other=cols.options[0]
    cols.value=other
    cols.value=current
    
output = wdg.interactive_output(graph, {'graphcolumns': cols})

display(cols, output)

Dropdown(description='Overview: ', options=('cases', 'hospital'), value='cases')

Output()

**Made by Jirapat Boonmee 220845469**
MSc Computing and Information Systems (conversion programme)